# Generating human 'attention' from source / summary input.

load the data

In [78]:
import json
import random
f = open('/Users/haldenl/nlpcapstone/data/attn_vis_data.json', 'r')
data = json.load(f)

print(data[0].keys())

dict_keys(['p_gens', 'article_lst', 'abstract_str', 'decoded_lst', 'attn_dists'])


process

In [79]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [80]:
import numpy as np
np.set_printoptions(threshold=np.inf)


softmax function, from @nolanbconaway

In [81]:
def softmax(X, theta = 1.0, axis = None):
    """
    Compute the softmax of each element along an axis of X.

    Parameters
    ----------
    X: ND-Array. Probably should be floats. 
    theta (optional): float parameter, used as a multiplier
        prior to exponentiation. Default = 1.0
    axis (optional): axis to compute values along. Default is the 
        first non-singleton axis.

    Returns an array the same size as X. The result will sum to 1
    along the specified axis.
    """

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    # multiply y against the theta parameter, 
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis = axis), axis)

    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis = axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p

### Calculates the similarity betweens sentences in article and summary (softmaxed)

In [102]:
def getSentenceSimilarities(article_sentences, human_sentences):
    sentence_similarities = np.zeros([len(human_sentences), len(article_sentences)])
    
    
    for human_index, human_sent in enumerate(human_sentences):
        for article_index, article_sent in enumerate(article_sentences):
            a_sent = article_sent
            h_sent = human_sent
            
            similarity = h_sent.similarity(a_sent)
            sentence_similarities[human_index][article_index] = similarity
        
    sentence_similarities = softmax(sentence_similarities, theta=50, axis=1)
    return sentence_similarities
    

### Calculates 'attention' weights between article and summary

In [103]:
def getWeights(article, human):
    weights = np.zeros([len(human), len(article)])
    
    article_sentences = list(article.sents)
    human_sentences = list(human.sents)
        
    sentence_similarities = getSentenceSimilarities(article_sentences, human_sentences)
    
    count = 0
    human_rows = {}
    for human_index, human_sent in enumerate(human_sentences):
        human_rows[human_index] = {}
        for human_tok_index, human_token in enumerate(human_sent):
            human_rows[human_index][human_tok_index] = count
            count += 1
            
    count = 0
    article_columns = {}
    for article_index, article_sent in enumerate(article_sentences):
        article_columns[article_index] = {}
        for article_tok_index, article_token in enumerate(article_sent):
            article_columns[article_index][article_tok_index] = count
            count += 1
    
    for human_index, human_sent in enumerate(human_sentences):
        human_sent = nlp(human_sent.text)
        
        for article_index, article_sent in enumerate(article_sentences):
            article_sent = nlp(article_sent.text)
        
            sentence_sim = sentence_similarities[human_index][article_index]
            
            for human_tok_index, human_token in enumerate(human_sent):
                count += 1
                for article_tok_index, article_token in enumerate(article_sent):
                    similarity = 0
                    if (len(human_token.text) == 1 or len(article_token.text) == 1):
                        similarity = 0.1 * random.random()
                    else:
                        similarity = human_token.similarity(article_token)
                        
                        
                    weight = sentence_sim * similarity
    
                    row = human_rows[human_index][human_tok_index]
                    column = article_columns[article_index][article_tok_index]
                    weights[row][column] = weight                  
                  
    weights = softmax(weights, theta=100, axis=1)
    
    return weights

In [104]:
article_str="(CNN)Five Muslim women stand tall in a boxing ring, slowly winding strapping around their hands. A spotlight is trained on them and the audience draws close, hanging on their every word. \"How can we be truthful if we have to hide out of shame, feel humiliated for feeling, for saying, doing, thinking as we please? How can that be freedom, how can that be truth?\" Performed at Southbank Centre's WOW -- Women of the World Festival -- Common Wealth Theatre's \"No Guts, No Heart, No Glory,\" tells the true story of Muslim female boxers from Bradford in the north of England, and how they are using the sport to transcend religious and gender barriers. The critically acclaimed show -- in which the women stamp, dance, shout, swear, punch, and even crowd surf -- was partly inspired by Ambreen Sadiq, a former UK national champion, and one of Britain's first Muslim female boxers. Her journey to success wasn't easy -- as a victim of bullying at the age of 13 she found respite in boxing after being introduced to it by her father and brother. \"I got bullied quite a lot growing up, my dad's Pakistani and my mum's Portuguese,\" Sadiq told CNN. \"I got stick from both sides, from white and black people. I felt like I wasn't accepted. I used to move schools so many times just to feel accepted. \"People would stay stuff about how I looked... everyone said that's a boy's sport, what are you doing that for? You're going to look like a man, like a transsexual. \"I was bullied from such a young age, I felt like it was never going to stop. Boxing helped me to control that anger.\" It was a transformative experience for Sadiq -- she went from being quiet, having few friends and getting into fights to plucking up courage to start conversations, and thereby build self confidence. \"Instead of punching someone in the face I'd punch the bag,\" she added. \"If anyone came up to me and said anything I could just walk away from it -- whereas before I'd just straight punch them in the face or something. So it's helped me a lot with discipline.\" It's sometimes hard to imagine Sadiq as a boxer -- her movements are calm and assured, and a kind smile is never far from her lips. \"Boxing has made me the person I am today, and I'm so proud of the person I am,\" said the 20-year-old trailblazer, who is now looked up to by girls from all kinds of communities wanting to follow in her footsteps. \"I've had a lot of girls -- not just Asian girls -- coming up to me and asking, how do you stop people saying it's a boy's sport? How can you convince your parents to let you box?\" While boxing for many is just a sport -- albeit a violent one -- when it comes to women and Muslims participating it can feel more like a cause, with Sadiq often finding herself explaining her motivation to girls. \"What's the difference if a boy boxes or if a girl boxes? When we're in that ring we look like boxers -- we don't look like a girl or a boy. \"You come into that gym, you come into that show, you come into that ring thinking I'm a boxer, not that I'm a girl or I'm a boy.\" The role of Nicola Adams -- the first ever female boxer to win an Olympic gold medal -- has also been key in the fight for acceptance. Sadiq tells the story of how several years ago her coach Alwyn Belcher, who also worked with Adams, originally refused to train women -- that is, until he turned up to coach the GB team (believing they'd be men) only to find an enthusiastic bunch of women ready to work. \"That's the day he met Nicola Adams,\" recalls Sadiq with a laugh. \"He said from that day he thought girls were better boxers -- because they like to listen.\" Since Adams' success at the Olympic Games, females boxing once a week increased by 79% according to a 2013 Sport England study, but even the production of \"No Guts, No Heart, No Glory\" highlighted the barriers women potentially face. Cast member and aspiring medical student Freyaa Ali said there was a girl who had wanted to participate in the show, but was ultimately prevented from doing so by her relatives. \"It wasn't even the boxing and it wasn't even the speech, it was the dancing we do in the ring. I'm incredibly lucky to be able to do this, and I was just so sad for her. You know there are certain women out there who can't do what we're doing right now.\" \"I think that's what makes me even more passionate about doing this -- we want to give them a voice and give them a reason, so they know they can have confidence to do stuff like this.\" If women do feel restricted by their religion or gender, Ali urged them to take things step by step. \"Go for it, just go for it. I think maybe talk to your parents first, and just tell them Mum, Dad, this is what I want to do, I want to box, I want to dance, I want to act, or whatever she wants to do. \"Tell them, you know what? Its OK what I'm doing, I'm not going to be going out of my boundaries and doing bad things, I'm going to stick to my limits and stick to my religion, it's not harming anyone. \"My parents would say, if you're happy, we're happy. I think anyone's parents would say that wouldn't they? \"The first step is trying it and opening that door. Once you can open it an inch you can open it a mile.\""
human_str="\"No Guts, No Heart, No Glory\" is a play by Common Wealth Theatre that tells the true story of Muslim female boxers The performance was partly inspired by former UK national champion Ambreen Sadiq who experienced bullying and discrimination \"We want to give them a voice and give them a reason, so they know they can have confidence\" says cast member Freyaa Ali about girls in boxing"


In [ ]:
article = nlp(article_str.lower())  
human = nlp(human_str.lower())

weights = getWeights(article, human)

attentionRecords = []
inputRecords = []
outputRecords = []

for human_index, human_token in enumerate(human):
    for article_index, article_token in enumerate(article):
        
        attn_weight = weights[human_index][article_index]
                
        attentionRecords.append({
            'inputIndex': article_index,
            'outputIndex': human_index,
            'weight': attn_weight
        })
        
for human_index, human_token in enumerate(human):
    outputRecords.append({
        'index': human_index,
        'token': human_token.text
    })
    
for article_index, article_token in enumerate(article):
    inputRecords.append({
        'index': article_index,
        'token': article_token.text
    })
            
output = {
    'attentionRecords': attentionRecords,
    'inputTokens': inputRecords,
    'outputTokens': outputRecords
}

with open('/Users/haldenl/nlpcapstone/data/hierarchical_similarity_data_{0}.json'.format('women'), 'w') as out:
    json.dump(output, out, indent=2)
